In [3]:
import os
import matplotlib.pyplot as plt
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow.compat.v1 as tf
from keras import backend as K
import keras.layers as ly
from keras.models import Model, load_model

In [6]:
def yolo_filter_boxes(box_conf, boxes, box_class_probs, threshold = 0.6):
    box_scores = np.multiply(box_conf, box_class_probs)
    box_classes = K.agmax(box_scores, axis=-1)
    box_class_scores = K.max(box_scores, axis=-1)
    
    filtering_max = box_class_scores >= threshold
    
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    
    return scores, boxes, classes

In [9]:
with tf.Session() as test_a:
    box_conf = tf.random_normal([19, 19, 5, 1], mean=1, stddev=4)
    boxes = tf.random_normal([19, 19, 5, 4], mean=1, stddev=4)
    box_class_probs = tf.random_normal([19, 19, 5, 80], mean=1, stddev=4)
    scores, boxes, classes = yolo_filter_boxes(box_conf, boxes, box_class_probs, threshold=0.5)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.shape))
    print("boxes.shape = " + str(boxes.shape))
    print("classes.shape = " + str(classes.shape))

NotImplementedError: Cannot convert a symbolic Tensor (random_normal_9:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [12]:
def iou(box1, box2):
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = max(box1[2], box2[2])
    yi2 = max(box1[3], box2[3])
    inter_width = max(xi2 - xi1, 0)
    inter_height = max(yi2 - yi1, 0)
    inter_area = inter_width * inter_height
    
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])    
    union_area = box1_area + box2_area - inter_area
    
    iou = inter_area / union_area
    
    return iou

In [14]:
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4) 
print("iou for intersecting boxes = " + str(iou(box1, box2)))

box1 = (1,1,3,3)
box2 = (2,3,3,4)
print("iou for boxes that only touch at edges = " + str(iou(box1,box2)))

iou for intersecting boxes = 1.0
iou for boxes that only touch at edges = 0.25


In [15]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes=10, iou_threshold=0.5):
    max_boxes_tensor = K.variable(max_boxes, dtype="int32")
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_threshold)
    
    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)
    
    return scorers, boxes, classes

In [17]:
with tf.Session() as test_b:
    scores = tf.random_normal([54,], mean=1, stddev=4)
    boxes = tf.random_normal([54, 4], mean=1, stddev=4)    
    classes = tf.random_normal([54,], mean=1, stddev=4)   
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print('classes.shape = ' + str(classes.eval().shape))

AttributeError: module 'keras.backend' has no attribute 'get_session'

In [18]:
def yolo_eval(yolo_outputs, image_shape=(720, 1280), max_boxes=10, scores_threshold=0.6,
             iou_threshold=0.5):
    box_conf, box_xy, box_wh, box_class_probs = yolo_outputs
    
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    
    scores, boxes, classes = yolo_filter_boxes(box_conf, boxes, box_class_pobs)
    
    boxes = scale_boxes(boxes, image_shape)
    
    scores, boxes, classes = ylo_non_max_suppression(scores, boxes, classes, max_boxes=max_boxes,
                                                    iou_threshold=iou_threshold)
    return scores, boxes, classes

In [20]:
with tf.Session() as test_b:
    yolo_outputs = (tf.random_normal([19, 19, 5, 1], mean=1, stddev=4),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4),
                    tf.random_normal([19, 19, 5, 80], mean=1, stddev=4))
    scores, boxes, classes = yolo_eval(yolo_outputs)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

NameError: name 'yolo_boxes_to_corners' is not defined

In [21]:
sess = K.get_session()

AttributeError: module 'keras.backend' has no attribute 'get_session'

In [ ]:
def read_classes():
    